In [28]:
# 형법 pdf
file_path = './dataset/criminal-law.pdf'

# Langchain의 PyPDFLoader를 이용
from langchain_community.document_loaders import PyPDFLoader
import re

# PDF 로더 객체 생성
loader = PyPDFLoader(file_path)

# pdf의 각 페이지를 저장
pages = []

# 비동기 방식으로 pdf 페이지 로드
async for page in loader.alazy_load():
    pages.append(page)

In [29]:
# 맨앞 두 페이지는 쓸모없어서 버림
pages = pages[2:]

# 페이지 내용을 하나의 문자열로
full_text = "\n".join(page.page_content for page in pages)

In [30]:
# "\n제 n장" 패턴으로 텍스트 분할
split_parts = re.split(r'(\n제\d+장)', full_text)

In [31]:
from langchain_core.documents import Document

In [32]:
chunks = []
current_chunck_text = ""
# 첫 부분 처리 (첫 장 이전에 내용이 있을 경우)
if split_parts[0].strip():
    # 첫 부분은 특정 "장" 구분 기호가 없으므로 별도 처리
    # 이 부분에 해당하는 페이지 메타데이터를 찾아야 함 (여기서는 단순화하여 첫 페이지 메타데이터 사용 가정)
    # 실제로는 이 텍스트가 어느 페이지에 걸쳐 있는지 정확히 파악해야 함
    first_page_meta = pages[0].metadata if pages else {}
    chunks.append(Document(page_content=split_parts[0].strip(), metadata=first_page_meta))
    current_chunck_text = "" # 다음 장부터 시작하도록 초기화

In [33]:
# 분할된 부분을 순회하며 청크 생성
for i in range(1, len(split_parts), 2):
    # 구분 기호 (e.g., "\n 제 1장")와 내용 결합
    chapter_title = split_parts[i].strip()
    content = split_parts[i+1].strip()
    chunk_text = chapter_title + "\n" + content
    
    metadata = {"chapter": chapter_title}
    chunks.append(Document(page_content=chunk_text, metadata=metadata))

In [34]:
print(f"Total chunks created: {len(chunks)}")
if chunks:
    print("\n--- First Chunk ---")
    print(f"Metadata: {chunks[0].metadata}")
    print(f"Content:\n{chunks[0].page_content[:500]}...") # Print first 500 chars

if len(chunks) > 1:
    print("\n--- Second Chunk ---")
    print(f"Metadata: {chunks[1].metadata}")
    print(f"Content:\n{chunks[1].page_content[:500]}...") # Print first 500 chars

Total chunks created: 1

--- First Chunk ---
Metadata: {'producer': 'iText 2.1.7 by 1T3XT', 'creator': 'PyPDF', 'creationdate': '2013-11-08T14:53:09+09:00', 'moddate': '2013-11-08T14:53:09+09:00', 'source': './dataset/criminal-law.pdf', 'total_pages': 50, 'page': 2, 'page_label': '3'}
Content:
법제처                                                            3                                                       국가법령정보센터
「형법」
 
                    제1편 총칙
                       제1장 형법의 적용범위
 
제1조(범죄의 성립과 처벌) ①범죄의 성립과 처벌은 행위 시의 법률에 의한다.
②범죄 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하거나 형이 구법보다 경한
때에는 신법에 의한다.
③재판확정 후 법률의 변경에 의하여 그 행위가 범죄를 구성하지 아니하는 때에는 형의 집행
을 면제한다.
 
제2조(국내범) 본법은 대한민국영역 내에서 죄를 범한 내국인과 외국인에게 적용한다.
 
제3조(내국인의 국외범) 본법은 대한민국영역 외에서 죄를 범한 내국인에게 적용한다.
 
제4조(국외에 있는 내국선박 등에서 외국인이 범한 죄) 본법은 ...
